## 주식 가격 (대성에너지)

### 과거 데이터 일괄로 가져와서 저장

In [1]:
stock_code = '117580'   # 대성에너지 stock code

# 과거 기간
start_date = '20101201' # 2010년부터 (2010-12-24 일자부터 데이터 있음)
end_date = '20230630'   # 2023년 6월까지

#### 1. 주가 데이터 가져오기

In [2]:
from pykrx import stock

# 특정기간 특정 종목의 OHLCV
ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)

In [3]:
ohlcv.shape

(3085, 7)

In [4]:
# ohlcv

#### 2. 데이터 변환 및 전처리

In [5]:
df_ohlcv = ohlcv.reset_index()

In [6]:
ohlcv.shape

(3085, 7)

In [7]:
# df_ohlcv.dtypes

In [8]:
# 날짜 형식 변환 => 문자열 yyyymmdd
df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')

In [9]:
# df_ohlcv.dtypes

In [10]:
# df_ohlcv

#### 3. DB 테이블에 저장

In [11]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_ohlcv.itertuples():
    sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                            values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
conn.commit()
conn.close()